## 🐍 파이썬 텍스트 분석: Chapter 7. 비지도 학습 (토픽 모델링과 군집화)

이 노트북에서는 정해진 정답(레이블) 없이 데이터 자체의 특성만으로 숨겨진 구조를 찾아내는 **비지도 학습(Unsupervised Learning)** 기법을 배웁니다.

텍스트 데이터의 주요 주제들을 자동으로 추출하는 **토픽 모델링(Topic Modeling)** 과 유사한 문서들을 그룹으로 묶는 **군집화(Clustering)** 를 실습합니다.

---


### 🤔 비지도 학습(Unsupervised Learning)이란 무엇일까요?

**비지도 학습**은 말 그대로 '지도'(supervision)나 '정답'이 없는 데이터를 가지고 학습하는 방법입니다.

이해를 돕기 위해 쉬운 비유를 들어보겠습니다. 🍎🍇🍊

여러분 앞에 이름표가 없는 온갖 종류의 과일이 담긴 상자가 있다고 상상해 보세요. 사과, 오렌지, 포도, 바나나가 뒤죽박죽 섞여 있습니다. 여러분은 각 과일의 이름은 모르지만, **색깔, 모양, 크기**와 같은 특징을 보고 비슷한 것끼리 그룹으로 나눌 수는 있습니다.

* "동그랗고 빨간 것들은 이쪽에 모아두자." (아마도 사과 그룹)
* "길고 노란 것들은 저쪽에 모으자." (아마도 바나나 그룹)
* "작고 동그란 알맹이가 많이 달린 건 여기 두자." (아마도 포도 그룹)

이렇게 정답(과일 이름)을 모르는 상태에서 데이터의 **내재된 특성(색, 모양, 크기)**만을 이용해 스스로 패턴이나 구조를 찾아내 그룹으로 나누는 과정이 바로 **비지도 학습**입니다.

텍스트 분석에서의 비지도 학습도 마찬가지입니다. 수많은 뉴스 기사가 있을 때, 우리는 각 기사에 '스포츠', '정치', '경제' 같은 딱지를 붙여주지 않아도, 기사에 사용된 **단어들의 패턴**을 분석하여 비슷한 내용의 기사들을 그룹으로 묶거나(군집화), 전체 기사들을 관통하는 숨겨진 주제들(토픽 모델링)을 발견할 수 있습니다.

* **지도 학습(Supervised Learning)과의 차이점:** 지도 학습은 "이건 사과야", "이건 바나나야"라고 이름표가 붙은 과일 사진(정답 데이터)을 먼저 학습한 뒤, 새로운 과일 사진을 보고 이름을 맞추는 방식입니다. 반면 비지도 학습은 이름표 없이 비슷한 것끼리 분류하는 방식이죠.

이번 챕터에서는 텍스트 데이터의 숨겨진 구조를 파헤치는 두 가지 대표적인 비지도 학습 기법, **토픽 모델링**과 **군집화**를 실습해 보겠습니다.

---

### 💡 시작 전 준비: 토픽 분석용 데이터셋 생성 및 전처리

토픽 모델링과 군집화는 여러 주제가 섞인 데이터에서 그 진가를 발휘합니다. 실습을 위해 다양한 분야의 뉴스 기사 제목으로 구성된 샘플 데이터셋을 만들고, **Kiwipiepy**를 사용하여 전처리합니다.

In [2]:
# 필요 라이브러리 설치
# !pip install kiwipiepy scikit-learn pandas plotly pyldavis

import pandas as pd
import re
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 여러 주제를 가진 샘플 뉴스 데이터
raw_news_data = {
    'category': ['IT', 'IT', 'IT', 'IT', '정치', '정치', '정치', '스포츠', '스포츠', '스포츠', '경제', '경제'],
    'document': [
        '과기부, 데이터-AI 기반 신규 서비스 개발 사업 공모',
        '구글, 차세대 인공지능 모델 제미나이 프로 공개',
        '오픈AI 라이벌 등장, 미스트랄AI 거대언어모델 공개',
        '네이버, 생성형 AI 하이퍼클로바X 기술 컨퍼런스 개최',
        '여야, 내년도 예산안 처리 막판 협상 돌입',
        '대통령, 국무회의 주재하며 민생 안정 대책 논의',
        '국회, 본회의 열어 법안 처리 예정',
        '손흥민, 리그 10호골 기록하며 팀 승리 이끌어',
        '프로농구 플레이오프, 4강 대진표 확정',
        '이정후, 메이저리그 샌프란시스코 자이언츠와 계약',
        '한국은행, 기준금리 3.5%로 동결 결정',
        '코스피, 외인 매수세에 힘입어 소폭 상승 마감'
    ]
}
news_df = pd.DataFrame(raw_news_data)


In [3]:
kiwi = Kiwi()
# kiwi.tokenize를 사용하여 명사를 추출합니다.
# 토큰(token) 객체에서 품사(tag)가 'NNG'(일반명사), 'NNP'(고유명사)인 경우만 선택합니다.
def news_preprocess(text):
    text = re.sub(r'[^가-힣\s]', '', str(text))
    tokens = kiwi.tokenize(text)
    # 명사만 추출하여 토픽 분석에 더 적합하게 만듭니다.
    nouns = [token.form for token in tokens if token.tag in ['NNG', 'NNP']]
    return ' '.join([word for word in nouns if len(word) > 1])

news_df['processed'] = news_df['document'].apply(news_preprocess)

벡터화

In [4]:
# 1. LDA 토픽 모델링을 위한 DTM(단어 빈도 행렬) 생성
# 너무 자주 등장하거나(max_df) 너무 드물게 등장하는(min_df) 단어는 제외합니다.
count_vectorizer = CountVectorizer(max_df=0.85, min_df=2)
dtm = count_vectorizer.fit_transform(news_df['processed'])


In [5]:
# 2. 군집화를 위한 TF-IDF 행렬 생성
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(news_df['processed'])

print("데이터 준비 및 벡터화 완료.")
print("DTM 형태:", dtm.shape)
print("TF-IDF 행렬 형태:", tfidf_matrix.shape)
print("\n[전처리 후 데이터 샘플]")
print(news_df[['document', 'processed']].head())

데이터 준비 및 벡터화 완료.
DTM 형태: (12, 5)
TF-IDF 행렬 형태: (12, 5)

[전처리 후 데이터 샘플]
                         document                  processed
0  과기부, 데이터-AI 기반 신규 서비스 개발 사업 공모  기부 데이터 기반 신규 서비스 개발 사업 공모
1      구글, 차세대 인공지능 모델 제미나이 프로 공개   구글 차세대 인공지능 모델 제미나 프로 공개
2   오픈AI 라이벌 등장, 미스트랄AI 거대언어모델 공개  오픈 라이벌 등장 미스트 거대 언어 모델 공개
3  네이버, 생성형 AI 하이퍼클로바X 기술 컨퍼런스 개최   네이버 생성 하이퍼 클로 기술 컨퍼런스 개최
4         여야, 내년도 예산안 처리 막판 협상 돌입        내년도 예산안 처리 막판 협상 돌입


---

### 1. 토픽 모델링 (Topic Modeling) with LDA

#### 💡 개념 (Concept)

**토픽 모델링** 은 대량의 문서 집합에서 숨겨진 주요 주제(Topic)들을 자동으로 발견하는 기술입니다. 

가장 널리 사용되는 알고리즘 중 하나인 **LDA(Latent Dirichlet Allocation, 잠재 디리클레 할당)** 는 다음과 같은 가정에 기반합니다.

* **모든 문서는 여러 토픽의 혼합으로 구성되어 있다.** (예: 한 IT 기사는 70%의 'AI' 토픽과 30%의 '기업' 토픽으로 이루어짐)
* **모든 토픽은 여러 단어의 혼합으로 구성되어 있다.** (예: 'AI' 토픽은 '인공지능', '데이터', '모델' 등의 단어를 높은 확률로 포함함)

LDA는 이 가정을 바탕으로, 각 문서가 어떤 토픽 분포를 가지는지, 그리고 각 토픽이 어떤 단어 분포를 가지는지를 역으로 추적합니다. 

우리는 모델에 전체 토픽의 개수(`n_components`)를 미리 알려주어야 합니다.

#### 💻 예시 코드 (Example Code)

In [6]:
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델 생성 및 학습
# n_components는 추출할 토픽의 개수를 의미합니다.
# 데이터의 실제 카테고리 수인 4개로 설정해봅니다.
lda_model = LatentDirichletAllocation(n_components=4, random_state=42)
lda_model.fit(dtm) # LDA는 TF-IDF가 아닌 DTM(단어 빈도)을 입력으로 받습니다.

def display_topics(model, feature_names, n_top_words):
    """LDA 모델의 토픽별 상위 단어를 출력하는 함수"""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx+1}:", " ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

# 토픽별 상위 5개 단어 출력
n_top_words = 5
feature_names = count_vectorizer.get_feature_names_out()
print("LDA 토픽별 주요 단어:")
display_topics(lda_model, feature_names, n_top_words)

LDA 토픽별 주요 단어:
Topic #1: 모델 공개 프로 리그 처리
Topic #2: 처리 프로 리그 모델 공개
Topic #3: 리그 프로 처리 모델 공개
Topic #4: 프로 리그 처리 모델 공개


#### ✏️ 연습 문제 (Practice Problems)

1.  `LatentDirichletAllocation` 모델을 `n_components=3`으로 설정하여 다시 학습시키고, 토픽별 주요 단어의 변화를 관찰해 보세요. 토픽의 개수가 줄어들면서 각 토픽이 어떻게 합쳐지거나 재구성되었나요?


In [ ]:
# 코드 작성

2.  위 예제에서 `n_components=4`로 학습한 결과(4개의 토픽)를 보고, 각 토픽에 어울리는 이름(예: 'IT/기술', '정치/사회')을 직접 붙여보세요.

In [ ]:
# 코드 작성


---

### 2. LDA 결과 시각화 (Visualizing LDA with pyLDAvis)

#### 💡 개념 (Concept)

단어 목록만으로 토픽을 해석하는 것은 때로 어렵습니다. **pyLDAvis**는 LDA 모델의 결과를 인터랙티브 시각화로 보여주는 강력한 도구입니다. 이 시각화는 다음 정보를 제공합니다.

* **좌측 (토픽 분포도)**: 각 원은 하나의 토픽을 의미하며, 원의 크기는 해당 토픽의 비중을 나타냅니다. 원 사이의 거리는 토픽 간의 유사도를 보여줍니다. (가까울수록 유사)
* **우측 (단어 막대그래프)**: 특정 토픽을 선택했을 때, 해당 토픽을 구성하는 주요 단어들과 그 중요도를 보여줍니다.

#### 💻 예시 코드 (Example Code)

In [ ]:
 !pip install pyldavis

In [7]:
import pyLDAvis
import pyLDAvis.lda_model

# pyLDAvis를 위한 데이터 준비
# 이전에 n_components=4로 학습한 lda_model, dtm, count_vectorizer를 사용합니다.
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda_model, dtm, count_vectorizer)
pyLDAvis.display(vis)

#### ✏️ 연습 문제 (Practice Problems)

1.  위에서 생성된 `pyLDAvis` 시각화 결과에서, 1번 토픽(Topic 1) 원을 클릭해 보세요. 우측 막대그래프에 나타나는 주요 단어들은 무엇인가요? 

> 작성해보기

  2.  우측 상단의 슬라이더 `λ`(람다) 값을 0에 가깝게, 그리고 1에 가깝게 조절해 보세요. `λ=0`일 때와 `λ=1`일 때 나타나는 단어 목록의 특징은 각각 무엇인지 설명해 보세요. (힌트: `λ=0`은 토픽 내 단어 빈도, `λ=1`은 토픽 고유 단어에 가중치를 둡니다.)

> 작성해보기


---

### 3. 텍스트 군집화 (Text Clustering) with K-Means

#### 💡 개념 (Concept)

**군집화(Clustering)** 는 레이블이 없는 문서들을 내용의 유사도에 따라 여러 개의 그룹(군집, Cluster)으로 묶는 작업입니다. **K-평균(K-Means)** 알고리즘은 가장 대중적인 군집화 방법 중 하나입니다.

1.  먼저, 우리가 지정한 K개의 임의의 중심점(Centroid)을 설정합니다.
2.  모든 문서는 K개의 중심점 중 가장 가까운 곳에 소속됩니다.
3.  각 군집의 평균 위치로 중심점을 이동시킵니다.
4.  중심점의 위치에 더 이상 변화가 없을 때까지 2~3번 과정을 반복합니다.

이때 문서 간의 거리는 보통 TF-IDF 벡터를 이용하여 계산합니다.


#### 💻 예시 코드 (Example Code)

In [8]:
from sklearn.cluster import KMeans

# K-Means 군집화 수행
# n_clusters는 생성할 군집의 개수입니다. 샘플 데이터의 카테고리 수와 동일하게 4로 설정합니다.
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto') # n_init 기본값이 'auto'로 변경되었습니다.
kmeans.fit(tfidf_matrix) # 군집화는 보통 TF-IDF 행렬을 사용합니다.

# 각 문서가 어떤 군집에 속하는지 확인
news_df['cluster_id'] = kmeans.labels_
print("문서별 군집 할당 결과:")
print(news_df[['document', 'category', 'cluster_id']].sort_values(by='cluster_id'))

문서별 군집 할당 결과:
                          document category  cluster_id
4          여야, 내년도 예산안 처리 막판 협상 돌입       정치           0
6              국회, 본회의 열어 법안 처리 예정       정치           0
0   과기부, 데이터-AI 기반 신규 서비스 개발 사업 공모       IT           1
3   네이버, 생성형 AI 하이퍼클로바X 기술 컨퍼런스 개최       IT           1
5       대통령, 국무회의 주재하며 민생 안정 대책 논의       정치           1
10          한국은행, 기준금리 3.5%로 동결 결정       경제           1
11       코스피, 외인 매수세에 힘입어 소폭 상승 마감       경제           1
1       구글, 차세대 인공지능 모델 제미나이 프로 공개       IT           2
2    오픈AI 라이벌 등장, 미스트랄AI 거대언어모델 공개       IT           2
8            프로농구 플레이오프, 4강 대진표 확정      스포츠           2
7       손흥민, 리그 10호골 기록하며 팀 승리 이끌어      스포츠           3
9       이정후, 메이저리그 샌프란시스코 자이언츠와 계약      스포츠           3


#### ✏️ 연습 문제 (Practice Problems)

1.  `KMeans` 모델을 `n_clusters=3`으로 설정하여 다시 군집화를 수행해 보세요. 기존의 4개 카테고리가 3개의 군집으로 어떻게 재편성되었는지 결과를 확인하고 분석해 보세요.

In [9]:
# 코드 작성

2.  각 군집의 중심(centroid)에 가장 가까운 단어들을 찾아, 각 군집의 특징을 파악해 보세요. (힌트: `kmeans.cluster_centers_`는 각 군집의 중심 벡터입니다. `argsort()`를 사용하여 각 중심 벡터에서 값이 가장 큰 단어의 인덱스를 찾고, 이를 `tfidf_vectorizer.get_feature_names_out()`에 매핑할 수 있습니다.)


In [ ]:
# 코드 작성

---

### 4. 군집 결과 시각화 (Visualizing Clustering Results)

#### 💡 개념 (Concept)

수천 차원의 TF-IDF 벡터를 직접 시각화하는 것은 불가능합니다. 

따라서 **차원 축소(Dimensionality Reduction)** 기법을 사용하여 고차원 벡터를 2차원으로 압축한 뒤, 

산점도(Scatter Plot)로 시각화합니다. 

**t-SNE(t-Distributed Stochastic Neighbor Embedding)** 는 고차원 데이터의 지역적 구조를 잘 보존하면서 2차원으로 변환하는 비선형 차원 축소 기법으로, 특히 군집화 결과를 시각화할 때 매우 효과적입니다.

#### 💻 예시 코드 (Example Code)

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

# t-SNE를 사용하여 2차원으로 차원 축소
tsne = TSNE(n_components=2, random_state=42, perplexity=3)
tsne_components = tsne.fit_transform(tfidf_matrix.toarray())

# 데이터프레임에 t-SNE 결과 추가
news_df['tsne_x'] = tsne_components[:, 0]
news_df['tsne_y'] = tsne_components[:, 1]

# 인터랙티브 산점도 시각화
fig = px.scatter(news_df,
                 x='tsne_x',
                 y='tsne_y',
                 color='cluster_id',
                 hover_data=['document', 'category'],
                 title='K-Means 군집화 결과 (2D t-SNE)',
                 labels={'tsne_x': 't-SNE Component 1', 'tsne_y': 't-SNE Component 2', 'cluster_id': '군집 ID'}
)
fig.show()

#### ✏️ 연습 문제 (Practice Problems)

1.  PCA 대신 또 다른 차원 축소 기법인 **t-SNE** (`sklearn.manifold.TSNE`)를 사용하여 2차원으로 축소하고, 그 결과를 산점도로 시각화해 보세요. PCA 결과와 어떤 차이가 있는지 비교해 보세요.


In [ ]:
# 코드 작성

2.  위 예시 코드의 `px.scatter`에서 `color` 인자를 `'cluster_id'` 대신 실제 카테고리인 `'category'`로 변경하여 시각화해 보세요. K-Means가 얼마나 실제 카테고리를 잘 분리해냈는지 시각적으로 확인할 수 있습니다.

In [ ]:
# 코드 작성


---


### 🏆 최종 실습 과제: 실제 도서 리뷰 데이터로 숨은 목소리 찾기

지금까지 배운 토픽 모델링과 군집화 기법을 실제 데이터에 적용하여 독자들이 남긴 리뷰 속에 숨겨진 다양한 목소리와 주제를 발견해 봅시다.

**과제 목표:**
교보문고의 특정 베스트셀러 도서에 대한 리뷰를 직접 수집(크롤링)하고, 비지도 학습을 통해 리뷰들을 주제별로 묶고(토픽 모델링), 비슷한 내용의 리뷰들을 그룹화(군집화)하여 인사이트를 도출합니다.

#### 📚 1단계: 데이터 수집 (웹 크롤링)

먼저 분석할 리뷰 데이터를 수집해야 합니다. 제가 알려드리는 방식으로 수집을 해보세요(별도 교육)

* **대상 도서:** 밑바닥부터 시작하는 딥러닝
* **대상 URL:** `https://product.kyobobook.co.kr/detail/S000001057805`
* **수집 내용:** 리뷰 텍스트

In [ ]:
# 웹 크롤링을 위한 예시 코드
import requests
import pandas as pd

# 리뷰 수집

# Kiwipiepy를 사용하여 명사만 추출합니다.
kiwi = Kiwi()


#### 🔍 2단계: 토픽 모델링 (LDA)으로 리뷰 주제 파악하기

수집한 리뷰들에는 어떤 숨겨진 주제들이 있을지 LDA를 통해 분석해 봅시다.

1.  **DTM 생성:** 전처리된 'processed' 데이터를 `CountVectorizer`를 사용하여 DTM(단어-문서 행렬)으로 변환하세요.
2.  **LDA 모델 학습:** `LatentDirichletAllocation`을 사용해 **4개의 토픽**을 추출해 보세요.
3.  **결과 해석:**
    * 각 토픽을 대표하는 상위 5~7개의 키워드를 출력하세요.
    * 키워드를 바탕으로 각 토픽에 **이름을 붙여보세요.** 예를 들어, "실천과 변화", "선물 및 추천", "번역 및 가독성" 등과 같이 해석할 수 있습니다. 이를 통해 독자들이 어떤 관점에서 이 책을 평가하는지 파악할 수 있습니다.


#### 🧩 3단계: K-Means 군집화로 유사 리뷰 그룹화하기

비슷한 내용을 담고 있는 리뷰들을 그룹으로 묶어 봅시다.

1.  **TF-IDF 행렬 생성:** 전처리된 'processed' 데이터를 `TfidfVectorizer`를 사용하여 TF-IDF 행렬로 변환하세요.
2.  **K-Means 모델 학습:** `KMeans`를 사용하여 **4개의 군집**으로 리뷰들을 나누세요.
3.  **결과 분석:**
    * 원본 `review_df`에 'cluster\_id' 컬럼을 추가하여 각 리뷰가 어떤 군집에 속하는지 확인하세요.
    * 각 군집별로 리뷰 내용을 몇 개씩 출력하여, 그룹이 어떤 기준으로 묶였는지(e.g., 긍정적 실천 후기, 책의 구성 칭찬, 배송 관련 등) 그 특징을 분석해 보세요.


#### 📊 4단계: 시각화로 군집 결과 확인하기

군집화 결과를 PCA나 t-SNE를 이용해 2차원 공간에 시각화하여 그룹이 잘 형성되었는지 확인합니다.

1.  **차원 축소:** `PCA`나 `t-SNE`를 사용해 TF-IDF 행렬을 2개의 주성분으로 축소하세요.
2.  **산점도 시각화:** `plotly.express`를 사용해 결과를 산점도로 그리세요.
    * 각 점의 색상은 `cluster_id`로 구분합니다.
    * 마우스를 점 위에 올렸을 때 원본 리뷰(`review`)가 표시되도록 설정하여, 각 군집의 분포와 특징을 시각적으로 탐색해 보세요.


#### ✨ 도전 과제

* 토픽과 군집의 개수(`n_components`, `n_clusters`)를 3, 5 등 다른 숫자로 변경하며 결과를 비교해 보세요. 어떤 개수가 가장 해석하기 좋은 결과를 도출하나요?
* 다른 책(예: 소설, 에세이)의 리뷰를 수집하여 동일한 분석을 수행하고, 책의 장르에 따라 리뷰의 주제와 군집이 어떻게 달라지는지 비교 분석해 보세요.